# App-1 : Le Probleme des N-Reines

**Navigation** : [Index](../README.md) | [Foundations](../Foundations/README.md) | [App-2 GraphColoring >>](App-2-GraphColoring.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Formaliser** le probleme des N-Reines comme un CSP (variables, domaines, contraintes) -- *Bloom : Comprendre*
2. **Implementer** un solveur par backtracking avec heuristiques (MRV, Forward Checking) -- *Bloom : Appliquer*
3. **Comparer** backtracking, min-conflicts et OR-Tools CP-SAT sur differentes tailles -- *Bloom : Analyser*
4. **Evaluer** les forces et limites de chaque approche selon la taille du probleme -- *Bloom : Evaluer*

### Prerequis
- Python 3.10+, matplotlib, numpy, ortools
- [Search-6 : CSP Fundamentals](../Foundations/Search-6-CSP-Fundamentals.ipynb) -- backtracking, MRV, formalisme CSP

### Duree estimee : 30 minutes

---

## 1. Introduction (~3 min)

Le probleme des **N-Reines** est l'un des benchmarks les plus celebres en informatique et en intelligence artificielle. L'enonce est simple :

> Placer $N$ reines sur un echiquier $N \times N$ de sorte qu'aucune paire de reines ne s'attaque mutuellement (ni meme ligne, ni meme colonne, ni meme diagonale).

### Historique

| Date | Auteur | Contribution |
|------|--------|--------------|
| 1848 | Max Bezzel | Pose le probleme pour N=8 dans un journal d'echecs |
| 1850 | Franz Nauck | Premiere solution complete et generalisation a N reines |
| 1874 | S. Gunther & J.W.L. Glaisher | Premiere approche systematique par determinants |
| 1992 | Sosic & Gu | Min-Conflicts : resolution quasi-lineaire en temps |

### Pourquoi ce probleme est important

- **Benchmark CSP** : il sert a evaluer les algorithmes de satisfaction de contraintes
- **Scalabilite** : l'espace de recherche croit exponentiellement ($N^N$ naif), mais des algorithmes intelligents resolvent $N = 10^6$ en secondes
- **Pedagogie** : il illustre parfaitement la difference entre recherche systematique et recherche locale

In [ ]:
# Imports pour tout le notebook
import sys
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Helpers partages de la serie Search
sys.path.insert(0, '..')
from search_helpers import benchmark_table, plot_benchmark

# Seed pour reproductibilite
random.seed(42)
np.random.seed(42)

print("Imports OK")

---

## 2. Formulation comme CSP (~5 min)

### Modelisation

La cle d'une resolution efficace est de choisir une bonne representation. Puisque chaque colonne doit contenir exactement une reine, on utilise :

| Composant CSP | Definition | Taille |
|---------------|------------|--------|
| **Variables** | $Q_0, Q_1, \ldots, Q_{N-1}$ (une par colonne) | $N$ |
| **Domaines** | $D_i = \{0, 1, \ldots, N-1\}$ (numero de ligne) | $N$ valeurs chacun |
| **Contraintes** | Pour tout $i \neq j$ : $Q_i \neq Q_j$ et $|i - j| \neq |Q_i - Q_j|$ | $\binom{N}{2}$ paires |

### Espace de recherche

| Representation | Taille | Explication |
|----------------|--------|-------------|
| Naive (case libre) | $\binom{N^2}{N}$ | Combinaisons de N cases parmi $N^2$ |
| Une reine par colonne | $N^N$ | Chaque colonne choisit une ligne |
| Permutation | $N!$ | Contrainte AllDifferent sur les lignes |

Pour $N = 8$ : $N^N = 16\,777\,216$ tandis que $N! = 40\,320$ -- un facteur de reduction de 416x rien qu'avec la modelisation.

### Contraintes detaillees

Pour deux reines aux colonnes $i$ et $j$ (avec $i < j$), il faut :

$$Q_i \neq Q_j \quad \text{(pas meme ligne)}$$
$$|Q_i - Q_j| \neq |i - j| \quad \text{(pas meme diagonale)}$$

La contrainte de colonne est deja satisfaite par construction (une variable par colonne).

Implementons d'abord une fonction de visualisation de l'echiquier, puis verifions visuellement une solution connue pour $N = 8$.

In [ ]:
def draw_queens(queens, n=None, title="Solution N-Reines", ax=None):
    """Visualise un placement de reines sur un echiquier.

    Args:
        queens: liste ou dict (col -> row). queens[col] = row.
        n: taille de l'echiquier (deduit de queens si None).
        title: titre du graphique.
        ax: axes matplotlib (cree une nouvelle figure si None).
    """
    if isinstance(queens, dict):
        queens_list = [queens[c] for c in sorted(queens.keys())]
    else:
        queens_list = list(queens)

    if n is None:
        n = len(queens_list)

    if ax is None:
        fig, ax = plt.subplots(figsize=(max(5, n * 0.7), max(5, n * 0.7)))
    else:
        fig = ax.figure

    # Dessiner l'echiquier
    for row in range(n):
        for col in range(n):
            color = '#F0D9B5' if (row + col) % 2 == 0 else '#B58863'
            rect = patches.Rectangle((col, n - 1 - row), 1, 1,
                                     facecolor=color, edgecolor='#8B7355',
                                     linewidth=0.5)
            ax.add_patch(rect)

    # Placer les reines
    for col, row in enumerate(queens_list):
        ax.text(col + 0.5, n - 1 - row + 0.5, '\u265B',
                ha='center', va='center',
                fontsize=max(8, 28 - n),
                color='#2C1810')

    # Lignes d'attaque (pour petits echiquiers)
    conflicts = []
    for i in range(len(queens_list)):
        for j in range(i + 1, len(queens_list)):
            ri, rj = queens_list[i], queens_list[j]
            if ri == rj or abs(ri - rj) == abs(i - j):
                conflicts.append((i, ri, j, rj))

    for ci, ri, cj, rj in conflicts:
        ax.plot([ci + 0.5, cj + 0.5],
                [n - 1 - ri + 0.5, n - 1 - rj + 0.5],
                'r-', linewidth=2, alpha=0.6)

    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_aspect('equal')
    ax.set_xticks([i + 0.5 for i in range(n)])
    ax.set_yticks([i + 0.5 for i in range(n)])
    ax.set_xticklabels(range(n), fontsize=9)
    ax.set_yticklabels(range(n - 1, -1, -1), fontsize=9)
    ax.set_xlabel('Colonne', fontsize=10)
    ax.set_ylabel('Ligne', fontsize=10)
    ax.set_title(title, fontsize=12, fontweight='bold')

    n_conflicts = len(conflicts)
    if n_conflicts > 0:
        ax.text(n / 2, -0.5, f'{n_conflicts} conflit(s) detecte(s)',
                ha='center', color='red', fontsize=10, fontweight='bold')

    return fig


# Afficher une solution connue pour N=8
known_solution_8 = [0, 4, 7, 5, 2, 6, 1, 3]
draw_queens(known_solution_8, title="Solution connue des 8-Reines")
plt.tight_layout()
plt.show()

### Interpretation : visualisation de la solution

**Sortie obtenue** : l'echiquier affiche 8 reines sans aucune ligne d'attaque rouge, confirmant que le placement `[0, 4, 7, 5, 2, 6, 1, 3]` est une solution valide.

| Verification | Resultat |
|-------------|----------|
| Lignes distinctes | Oui (0, 1, 2, 3, 4, 5, 6, 7 -- une permutation) |
| Colonnes distinctes | Oui (par construction : une reine par colonne) |
| Diagonales | Aucune paire en attaque diagonale |

> **Remarque** : pour $N = 8$, il existe exactement **92 solutions distinctes** (12 fondamentales sous symetries). Nous les enumererons dans la section CP-SAT.

---

## 3. Approche 1 : Backtracking (~7 min)

Le backtracking est la methode de reference pour les CSP. Nous allons implementer trois variantes de complexite croissante :

1. **Backtracking simple** : exploration brute, colonne par colonne
2. **Avec heuristique MRV** : choisir la colonne au domaine le plus restreint
3. **Avec Forward Checking** : propager les contraintes pour reduire les domaines

### 3.1 Backtracking simple

L'algorithme place une reine dans chaque colonne de gauche a droite. Pour chaque ligne candidate, il verifie si la reine est en conflit avec les reines deja placees.

In [ ]:
def is_safe(queens, col, row):
    """Verifie si placer une reine en (col, row) est compatible
    avec les reines deja placees dans queens[0..col-1]."""
    for c in range(col):
        r = queens[c]
        if r == row:                          # meme ligne
            return False
        if abs(c - col) == abs(r - row):      # meme diagonale
            return False
    return True


def backtracking_simple(n):
    """Backtracking simple pour N-Reines.

    Parcourt les colonnes de gauche a droite.
    Retourne (solution, nodes_explored).
    """
    queens = [0] * n
    nodes = [0]  # compteur mutable

    def solve(col):
        if col == n:
            return True
        for row in range(n):
            nodes[0] += 1
            if is_safe(queens, col, row):
                queens[col] = row
                if solve(col + 1):
                    return True
        return False

    if solve(0):
        return queens[:], nodes[0]
    return None, nodes[0]


# Test rapide sur N=8
sol, nodes = backtracking_simple(8)
print(f"Solution 8-Reines : {sol}")
print(f"Noeuds explores   : {nodes}")

### 3.2 Avec heuristique MRV

L'heuristique **MRV** (Minimum Remaining Values) choisit a chaque etape la variable (colonne) dont le domaine restant est le plus petit. C'est la strategie **fail-first** : on detecte les impasses le plus tot possible.

Pour N-Reines, cela signifie choisir la colonne qui a le moins de lignes viables.

In [ ]:
def backtracking_mrv(n):
    """Backtracking avec heuristique MRV pour N-Reines.

    Retourne (solution_dict, nodes_explored).
    """
    domains = {col: set(range(n)) for col in range(n)}
    assignment = {}
    nodes = [0]

    def get_viable(col):
        """Retourne les lignes viables pour une colonne."""
        viable = []
        for row in domains[col]:
            ok = True
            for c2, r2 in assignment.items():
                if r2 == row or abs(c2 - col) == abs(r2 - row):
                    ok = False
                    break
            if ok:
                viable.append(row)
        return viable

    def solve():
        if len(assignment) == n:
            return True

        # MRV : choisir la colonne avec le moins de valeurs viables
        unassigned = [c for c in range(n) if c not in assignment]
        col = min(unassigned, key=lambda c: len(get_viable(c)))

        for row in get_viable(col):
            nodes[0] += 1
            assignment[col] = row
            if solve():
                return True
            del assignment[col]
        return False

    if solve():
        result = [assignment[c] for c in range(n)]
        return result, nodes[0]
    return None, nodes[0]


sol_mrv, nodes_mrv = backtracking_mrv(8)
print(f"Solution (MRV)    : {sol_mrv}")
print(f"Noeuds explores   : {nodes_mrv}")

### 3.3 Avec Forward Checking

Le **Forward Checking** va plus loin que MRV : a chaque assignation, il propage immediatement les contraintes pour retirer des domaines des variables non assignees toute valeur rendue impossible. Si un domaine devient vide, on backtrack immediatement sans explorer le sous-arbre.

| Technique | Quand detecte-t-on l'echec ? |
|-----------|------------------------------|
| Backtracking simple | Au moment ou on essaie une valeur |
| Forward Checking | Des qu'un domaine voisin devient vide |

In [ ]:
def backtracking_fc(n):
    """Backtracking avec Forward Checking pour N-Reines.

    Retourne (solution, nodes_explored).
    """
    queens = [None] * n
    # domains[col] = ensemble des lignes encore possibles
    domains = [set(range(n)) for _ in range(n)]
    nodes = [0]

    def forward_check(col, row):
        """Retire les valeurs inconsistantes des domaines futurs.
        Retourne la liste des reductions effectuees (pour restauration)."""
        pruned = []
        for c2 in range(n):
            if queens[c2] is not None or c2 == col:
                continue
            # Retirer la meme ligne
            if row in domains[c2]:
                domains[c2].remove(row)
                pruned.append((c2, row))
            # Retirer les diagonales
            d = abs(c2 - col)
            for diag_row in [row - d, row + d]:
                if 0 <= diag_row < n and diag_row in domains[c2]:
                    domains[c2].remove(diag_row)
                    pruned.append((c2, diag_row))
        return pruned

    def restore(pruned):
        """Restaure les valeurs retirees."""
        for c, r in pruned:
            domains[c].add(r)

    def solve(col_idx):
        if col_idx == n:
            return True

        # MRV : choisir la colonne non assignee au domaine le plus petit
        unassigned = [c for c in range(n) if queens[c] is None]
        col = min(unassigned, key=lambda c: len(domains[c]))

        for row in list(domains[col]):
            nodes[0] += 1
            queens[col] = row
            old_domain = domains[col].copy()
            domains[col] = set()  # assignee

            pruned = forward_check(col, row)

            # Verifier qu'aucun domaine futur n'est vide
            empty = any(len(domains[c]) == 0
                        for c in range(n) if queens[c] is None)

            if not empty and solve(col_idx + 1):
                return True

            # Restauration
            restore(pruned)
            domains[col] = old_domain
            queens[col] = None

        return False

    if solve(0):
        return queens[:], nodes[0]
    return None, nodes[0]


sol_fc, nodes_fc = backtracking_fc(8)
print(f"Solution (FC+MRV) : {sol_fc}")
print(f"Noeuds explores   : {nodes_fc}")

### 3.4 Benchmark des variantes de backtracking

Comparons les trois variantes sur differentes tailles de probleme. Le nombre de noeuds explores et le temps d'execution mesurent l'efficacite.

In [ ]:
def run_benchmark_bt(sizes):
    """Benchmark des trois variantes de backtracking."""
    results = {name: {'nodes': [], 'times': []}
               for name in ['Simple', 'MRV', 'FC+MRV']}

    solvers = [
        ('Simple', backtracking_simple),
        ('MRV', backtracking_mrv),
        ('FC+MRV', backtracking_fc),
    ]

    print(f"{'N':>4}  {'Simple':>12} {'MRV':>12} {'FC+MRV':>12}   (noeuds explores)")
    print("-" * 60)

    for n in sizes:
        row = f"{n:>4}"
        for name, solver in solvers:
            t0 = time.time()
            sol, nodes = solver(n)
            elapsed = time.time() - t0
            results[name]['nodes'].append(nodes)
            results[name]['times'].append(elapsed * 1000)
            row += f"  {nodes:>12}"
        print(row)

    return results


bt_sizes = [4, 8, 12, 16]
bt_results = run_benchmark_bt(bt_sizes)

Tracons les resultats en echelle logarithmique pour mieux apprecier les differences entre variantes.

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Simple': '#2196F3', 'MRV': '#4CAF50', 'FC+MRV': '#FF9800'}

# Noeuds explores (echelle log)
for name in ['Simple', 'MRV', 'FC+MRV']:
    axes[0].plot(bt_sizes, bt_results[name]['nodes'],
                 'o-', label=name, color=colors[name], linewidth=2, markersize=8)
axes[0].set_xlabel('N (taille echiquier)', fontsize=11)
axes[0].set_ylabel('Noeuds explores', fontsize=11)
axes[0].set_title('Noeuds explores par variante', fontweight='bold')
axes[0].set_yscale('log')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Temps d'execution
for name in ['Simple', 'MRV', 'FC+MRV']:
    axes[1].plot(bt_sizes, bt_results[name]['times'],
                 'o-', label=name, color=colors[name], linewidth=2, markersize=8)
axes[1].set_xlabel('N (taille echiquier)', fontsize=11)
axes[1].set_ylabel('Temps (ms)', fontsize=11)
axes[1].set_title('Temps d\'execution par variante', fontweight='bold')
axes[1].set_yscale('log')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Benchmark backtracking -- N-Reines', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Interpretation : benchmark backtracking

**Sortie obtenue** : le nombre de noeuds et le temps augmentent exponentiellement avec $N$, mais les heuristiques reduisent fortement l'exploration.

| Variante | Principe | Impact sur N=16 |
|----------|----------|------------------|
| Simple | Ordre fixe, pas de propagation | Reference (le plus lent) |
| MRV | Fail-first : variable la plus contrainte d'abord | Reduction significative |
| FC+MRV | Propagation immediate + fail-first | Reduction maximale |

**Points cles** :
1. Forward Checking detecte les impasses plus tot en vidant les domaines
2. MRV guide la recherche vers les variables critiques
3. La combinaison FC+MRV est la plus efficace pour le backtracking

> **Limite** : meme avec FC+MRV, le backtracking reste exponentiel dans le pire cas. Pour de grands $N$, il faut une approche fondamentalement differente.

---

## 4. Approche 2 : Min-Conflicts (~7 min)

L'algorithme **Min-Conflicts** (Minton et al., 1992) est une methode de **recherche locale** pour les CSP. Son principe est radicalement different du backtracking :

1. **Demarrer** avec une assignation complete aleatoire (toutes les variables ont une valeur)
2. **Tant qu'il y a des conflits** :
   - Choisir aleatoirement une variable en conflit
   - Lui assigner la valeur qui **minimise** le nombre de conflits
3. **Repeter** jusqu'a trouver une solution ou atteindre un maximum d'iterations

### Pourquoi ca marche ?

Le resultat surprenant de Sosic & Gu (1990) est que pour les N-Reines, min-conflicts trouve une solution en temps **quasi-constant** par rapport a $N$ (pour des instances aleatoires). En partant d'un placement aleatoire sur un echiquier de taille $N$, on n'a besoin que de $O(N)$ reparations en moyenne.

| Aspect | Backtracking | Min-Conflicts |
|--------|-------------|---------------|
| Type | Systematique, complet | Recherche locale, incomplet |
| Garantie de solution | Oui (si elle existe) | Non (peut rester coince) |
| Complexite typique N-Reines | Exponentielle | Quasi-lineaire |
| Demarrage | Assignation vide | Assignation complete aleatoire |

In [ ]:
def count_conflicts(queens, col):
    """Nombre de reines attaquant la reine en colonne col."""
    n = len(queens)
    row = queens[col]
    conflicts = 0
    for c in range(n):
        if c == col:
            continue
        r = queens[c]
        if r == row or abs(c - col) == abs(r - row):
            conflicts += 1
    return conflicts


def total_conflicts(queens):
    """Nombre total de paires de reines en conflit."""
    n = len(queens)
    total = 0
    for i in range(n):
        for j in range(i + 1, n):
            ri, rj = queens[i], queens[j]
            if ri == rj or abs(ri - rj) == abs(i - j):
                total += 1
    return total


def min_conflicts(n, max_steps=None):
    """Algorithme Min-Conflicts pour N-Reines.

    Args:
        n: taille de l'echiquier.
        max_steps: maximum d'iterations (defaut: 5*n).

    Retourne (solution ou None, steps_used, conflict_history).
    """
    if max_steps is None:
        max_steps = max(5 * n, 1000)

    # Initialisation aleatoire : une reine par colonne, ligne aleatoire
    queens = list(range(n))
    random.shuffle(queens)

    conflict_history = [total_conflicts(queens)]

    for step in range(max_steps):
        # Trouver les colonnes en conflit
        conflicted = [c for c in range(n) if count_conflicts(queens, c) > 0]

        if not conflicted:
            return queens, step, conflict_history  # Solution trouvee

        # Choisir une colonne en conflit au hasard
        col = random.choice(conflicted)

        # Trouver la ligne qui minimise les conflits
        min_conf = n + 1
        best_rows = []
        for row in range(n):
            old_row = queens[col]
            queens[col] = row
            c = count_conflicts(queens, col)
            queens[col] = old_row
            if c < min_conf:
                min_conf = c
                best_rows = [row]
            elif c == min_conf:
                best_rows.append(row)

        # Assigner la meilleure ligne (aleatoire en cas d'egalite)
        queens[col] = random.choice(best_rows)
        conflict_history.append(total_conflicts(queens))

    return None, max_steps, conflict_history  # Pas de solution trouvee


# Test sur N=8
random.seed(42)
sol_mc, steps_mc, history_mc = min_conflicts(8)
print(f"Solution (min-conflicts, N=8) : {sol_mc}")
print(f"Iterations : {steps_mc}")
print(f"Conflits initiaux : {history_mc[0]}, finaux : {history_mc[-1]}")

### Le "miracle" de Min-Conflicts : scalabilite

Le resultat le plus remarquable de min-conflicts est sa capacite a resoudre des instances de **tres grande taille**. Testons sur $N = 8, 50, 100, 500, 1000$ et observons l'evolution du temps.

In [ ]:
# Scalabilite de min-conflicts
mc_sizes = [8, 50, 100, 500, 1000]
mc_results = []

print(f"{'N':>6}  {'Temps (ms)':>12}  {'Iterations':>12}  {'Trouve':>8}")
print("-" * 50)

for n in mc_sizes:
    random.seed(42)
    t0 = time.time()
    sol, steps, hist = min_conflicts(n, max_steps=10 * n)
    elapsed = (time.time() - t0) * 1000
    found = sol is not None
    mc_results.append({
        'n': n,
        'time_ms': elapsed,
        'steps': steps,
        'found': found
    })
    print(f"{n:>6}  {elapsed:>12.1f}  {steps:>12}  {'Oui' if found else 'Non':>8}")

Tracons l'evolution du temps et du nombre d'iterations en fonction de $N$.

In [ ]:
# Visualisation : temps vs N pour min-conflicts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ns = [r['n'] for r in mc_results]
times = [r['time_ms'] for r in mc_results]
steps_list = [r['steps'] for r in mc_results]

axes[0].plot(ns, times, 'o-', color='#E91E63', linewidth=2, markersize=8)
axes[0].set_xlabel('N (taille echiquier)', fontsize=11)
axes[0].set_ylabel('Temps (ms)', fontsize=11)
axes[0].set_title('Temps d\'execution de Min-Conflicts', fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ns, steps_list, 's-', color='#9C27B0', linewidth=2, markersize=8)
axes[1].set_xlabel('N (taille echiquier)', fontsize=11)
axes[1].set_ylabel('Iterations', fontsize=11)
axes[1].set_title('Iterations pour converger', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Scalabilite de Min-Conflicts', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Observons aussi la courbe de convergence des conflits au fil des iterations sur une instance de taille $N = 100$.

In [ ]:
# Courbe de convergence : evolution des conflits au fil des iterations (N=100)
random.seed(123)
sol_100, steps_100, history_100 = min_conflicts(100, max_steps=2000)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(history_100, color='#E91E63', linewidth=1.5)
ax.set_xlabel('Iteration', fontsize=11)
ax.set_ylabel('Nombre de conflits', fontsize=11)
ax.set_title(f'Convergence Min-Conflicts (N=100, resolu en {steps_100} iterations)',
             fontweight='bold')
ax.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='Zero conflits')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretation : scalabilite de Min-Conflicts

**Sortie obtenue** : min-conflicts resout le probleme des N-Reines en un temps qui croit bien plus lentement que le backtracking.

| N | Temps min-conflicts | Iterations | Observation |
|---|--------------------|-----------|--------------|
| 8 | < 1 ms | < 10 | Trivial |
| 50 | quelques ms | ~50 | Lineaire |
| 100 | quelques ms | ~100 | Lineaire |
| 1000 | dizaines de ms | ~1000 | Toujours lineaire |

**Points cles** :
1. Le nombre d'iterations croit quasi-lineairement avec $N$, pas exponentiellement
2. La courbe de convergence montre une decroissance rapide des conflits
3. Le temps par iteration est $O(N)$ (compter les conflits), donc le temps total est $O(N^2)$ en pratique

> **Pourquoi ca marche ?** L'espace des solutions des N-Reines est "dense" pour les instances aleatoires. En partant d'une permutation aleatoire, on est generalement proche d'une solution. Chaque reparation locale reduit les conflits sans en creer beaucoup de nouveaux.

> **Attention** : min-conflicts est **incomplet** -- il ne garantit pas de trouver une solution et ne peut pas prouver qu'il n'en existe pas.

---

## 5. Approche 3 : OR-Tools CP-SAT (~5 min)

**OR-Tools CP-SAT** est le solveur de programmation par contraintes de Google. Il combine propagation de contraintes, recherche avec apprentissage de clauses (clause learning), et parallelisme.

### Avantages d'un solveur industriel

| Fonctionnalite | Implementation manuelle | CP-SAT |
|----------------|------------------------|--------|
| Propagation | Forward Checking basique | AC, bounds consistency, specialisee |
| Apprentissage | Aucun | Clause learning (CDCL) |
| Parallelisme | Non | Multi-thread natif |
| Contraintes globales | A implementer | `AddAllDifferent`, etc. |
| Enumeration | A implementer | Callback de solutions |

In [ ]:
from ortools.sat.python import cp_model


def solve_nqueens_cpsat(n, enumerate_all=False, time_limit_s=30):
    """Resoudre N-Reines avec OR-Tools CP-SAT.

    Args:
        n: taille de l'echiquier.
        enumerate_all: si True, trouve toutes les solutions.
        time_limit_s: limite de temps en secondes.

    Retourne (solutions_list, solve_time_ms, status_name).
    """
    model = cp_model.CpModel()

    # Variables : queens[i] = ligne de la reine en colonne i
    queens = [model.NewIntVar(0, n - 1, f'q_{i}') for i in range(n)]

    # Contrainte 1 : toutes les lignes differentes
    model.AddAllDifferent(queens)

    # Contrainte 2 : toutes les diagonales montantes differentes
    # queens[i] + i doit etre different pour tout i
    model.AddAllDifferent([queens[i] + i for i in range(n)])

    # Contrainte 3 : toutes les diagonales descendantes differentes
    # queens[i] - i doit etre different pour tout i
    model.AddAllDifferent([queens[i] - i for i in range(n)])

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_s

    if enumerate_all:
        # Callback pour collecter toutes les solutions
        class SolutionCollector(cp_model.CpSolverSolutionCallback):
            def __init__(self, queens_vars):
                cp_model.CpSolverSolutionCallback.__init__(self)
                self.queens = queens_vars
                self.solutions = []

            def on_solution_callback(self):
                sol = [self.Value(q) for q in self.queens]
                self.solutions.append(sol)

        collector = SolutionCollector(queens)
        solver.parameters.enumerate_all_solutions = True
        t0 = time.time()
        status = solver.Solve(model, collector)
        elapsed = (time.time() - t0) * 1000
        status_name = solver.StatusName(status)
        return collector.solutions, elapsed, status_name
    else:
        t0 = time.time()
        status = solver.Solve(model)
        elapsed = (time.time() - t0) * 1000
        status_name = solver.StatusName(status)
        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            sol = [solver.Value(q) for q in queens]
            return [sol], elapsed, status_name
        return [], elapsed, status_name


# Resoudre N=8
solutions_8, time_8, status_8 = solve_nqueens_cpsat(8)
print(f"CP-SAT N=8 : {status_8}")
print(f"Solution    : {solutions_8[0] if solutions_8 else 'Aucune'}")
print(f"Temps       : {time_8:.1f} ms")

### Enumeration de toutes les solutions (N=8)

Un avantage majeur de CP-SAT est sa capacite a **enumerer toutes les solutions**. Pour $N = 8$, on sait qu'il en existe exactement 92.

In [ ]:
# Enumerer toutes les solutions pour N=8
all_solutions_8, time_enum, status_enum = solve_nqueens_cpsat(8, enumerate_all=True)
print(f"N=8 : {len(all_solutions_8)} solutions trouvees en {time_enum:.1f} ms")
print(f"Status : {status_enum}")

# Afficher les 4 premieres solutions
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    draw_queens(all_solutions_8[i], n=8, title=f"Solution {i+1}", ax=ax)
plt.suptitle(f'4 des {len(all_solutions_8)} solutions des 8-Reines',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

### Performance CP-SAT sur de grandes instances

Testons CP-SAT sur des tailles croissantes, jusqu'a $N = 1000$.

In [ ]:
# Benchmark CP-SAT sur differentes tailles
cpsat_sizes = [8, 50, 100, 500, 1000]
cpsat_results = []

print(f"{'N':>6}  {'Temps (ms)':>12}  {'Status':>12}")
print("-" * 35)

for n in cpsat_sizes:
    solutions, elapsed, status = solve_nqueens_cpsat(n, time_limit_s=60)
    cpsat_results.append({
        'n': n,
        'time_ms': elapsed,
        'found': len(solutions) > 0,
        'status': status
    })
    print(f"{n:>6}  {elapsed:>12.1f}  {status:>12}")

### Interpretation : OR-Tools CP-SAT

**Sortie obtenue** : CP-SAT resout le probleme des N-Reines de maniere fiable, avec des performances competitives.

| Fonctionnalite | Resultat |
|----------------|----------|
| Premiere solution (N=8) | Quasi-instantane |
| Toutes les solutions (N=8) | 92 solutions en quelques ms |
| Grande instance (N=1000) | En secondes grace a la propagation avancee |

**Avantages de CP-SAT** :
1. **Modelisation declarative** : on decrit les contraintes, le solveur choisit la strategie
2. **`AddAllDifferent`** : contrainte globale plus efficace que des contraintes binaires
3. **Enumeration complete** : capacite a trouver toutes les solutions
4. **Robustesse** : pas besoin de tuner les heuristiques manuellement

> **Compromis** : CP-SAT a un overhead de demarrage (creation du modele, compilation interne) qui le rend moins rapide que min-conflicts pour de tres grandes instances ou une seule solution suffit.

---

## 6. Comparaison et analyse (~3 min)

Consolidons les resultats des trois approches dans un benchmark comparatif.

In [ ]:
# Benchmark comparatif sur les tailles communes
comparison_sizes = [8, 50, 100]

print("Benchmark comparatif N-Reines")
print("=" * 72)
print(f"{'N':>4}  {'BT Simple':>12}  {'BT FC+MRV':>12}  {'Min-Conf.':>12}  {'CP-SAT':>12}  (ms)")
print("-" * 72)

comparison_results = []

for n in comparison_sizes:
    row_data = {'n': n}

    # Backtracking simple (limite a N <= 20 pour eviter des temps trop longs)
    if n <= 20:
        t0 = time.time()
        backtracking_simple(n)
        row_data['bt_simple'] = (time.time() - t0) * 1000
    else:
        row_data['bt_simple'] = None

    # Backtracking FC+MRV (limite a N <= 50)
    if n <= 50:
        t0 = time.time()
        backtracking_fc(n)
        row_data['bt_fc'] = (time.time() - t0) * 1000
    else:
        row_data['bt_fc'] = None

    # Min-Conflicts
    random.seed(42)
    t0 = time.time()
    min_conflicts(n)
    row_data['mc'] = (time.time() - t0) * 1000

    # CP-SAT
    _, elapsed_cp, _ = solve_nqueens_cpsat(n)
    row_data['cpsat'] = elapsed_cp

    comparison_results.append(row_data)

    bt_s = f"{row_data['bt_simple']:.1f}" if row_data['bt_simple'] is not None else "--"
    bt_f = f"{row_data['bt_fc']:.1f}" if row_data['bt_fc'] is not None else "--"
    print(f"{n:>4}  {bt_s:>12}  {bt_f:>12}  {row_data['mc']:>12.1f}  {row_data['cpsat']:>12.1f}")

print("=" * 72)
print("(-- = trop lent pour cette taille)")

Visualisons la comparaison des temps d'execution sur une plage etendue de tailles, en echelle log-log.

In [ ]:
# Graphique comparatif sur les tailles gerees par toutes les approches
fig, ax = plt.subplots(figsize=(10, 6))

# Min-Conflicts et CP-SAT sur toute la plage
common_sizes = [8, 50, 100, 500, 1000]
mc_times = []
cpsat_times = []

for n in common_sizes:
    random.seed(42)
    t0 = time.time()
    min_conflicts(n)
    mc_times.append((time.time() - t0) * 1000)

    _, elapsed_cp, _ = solve_nqueens_cpsat(n)
    cpsat_times.append(elapsed_cp)

ax.plot(common_sizes, mc_times, 'o-', label='Min-Conflicts',
        color='#E91E63', linewidth=2, markersize=8)
ax.plot(common_sizes, cpsat_times, 's-', label='CP-SAT',
        color='#673AB7', linewidth=2, markersize=8)

# Backtracking FC+MRV (petites tailles uniquement)
bt_sizes_small = [8, 12, 16, 20]
bt_times_small = []
for n in bt_sizes_small:
    t0 = time.time()
    backtracking_fc(n)
    bt_times_small.append((time.time() - t0) * 1000)

ax.plot(bt_sizes_small, bt_times_small, '^-', label='Backtracking FC+MRV',
        color='#FF9800', linewidth=2, markersize=8)

ax.set_xlabel('N (taille echiquier)', fontsize=12)
ax.set_ylabel('Temps (ms)', fontsize=12)
ax.set_title('Comparaison des 3 approches -- N-Reines', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretation : comparaison finale

**Sortie obtenue** : les trois approches ont des profils de performance tres differents.

| Approche | N=8 | N=100 | N=1000 | Completude | Enumeration |
|----------|-----|-------|--------|------------|-------------|
| Backtracking FC+MRV | Rapide | Lent | Intraitable | Oui | Oui (lent) |
| Min-Conflicts | Rapide | Rapide | Rapide | Non | Non |
| CP-SAT | Rapide | Rapide | Rapide | Oui | Oui |

### Quand utiliser quelle approche ?

| Situation | Approche recommandee | Raison |
|-----------|---------------------|--------|
| Petit probleme (N < 20) | Backtracking FC+MRV | Complet, simple a implementer |
| Grande instance, une solution | Min-Conflicts | Quasi-lineaire, tres rapide |
| Toutes les solutions | CP-SAT | Enumeration + optimisation |
| Contraintes supplementaires | CP-SAT | Modelisation declarative flexible |
| Prototype pedagogique | Backtracking | Comprendre les mecanismes |

### Liens avec les Foundations

| Concept applique | Notebook de reference |
|-----------------|----------------------|
| Backtracking CSP, MRV, LCV | [Search-6 : CSP Fundamentals](../Foundations/Search-6-CSP-Fundamentals.ipynb) |
| Forward Checking, Arc Consistency | [Search-7 : CSP Consistency](../Foundations/Search-7-CSP-Consistency.ipynb) |
| CP-SAT, contraintes globales | [Search-8 : CSP Advanced](../Foundations/Search-8-CSP-Advanced.ipynb) |

---

## 7. Exercices

### Exercice 1 : le probleme des N-Tours

**Enonce** : adaptez le probleme pour placer $N$ **tours** sur un echiquier $N \times N$. Les tours attaquent en ligne et en colonne (mais pas en diagonale).

1. Quelle contrainte disparait par rapport aux N-Reines ?
2. Combien de solutions existe-t-il pour $N = 8$ ?
3. Implementez un solveur et verifiez votre reponse.

In [ ]:
# Exercice 1 : N-Tours
# La contrainte de diagonale disparait. Les solutions sont les permutations
# de {0, ..., N-1}, donc il y en a N! = 40320 pour N=8.

# A COMPLETER
# def solve_nrooks_cpsat(n):
#     model = cp_model.CpModel()
#     rooks = [model.NewIntVar(0, n - 1, f'r_{i}') for i in range(n)]
#     model.AddAllDifferent(rooks)
#     # Pas de contrainte de diagonale !
#     ...
#     return solutions_count

<details>
<summary><b>Indice exercice 1</b></summary>

Le probleme des N-Tours se reduit a compter les permutations. La seule contrainte est `AddAllDifferent(rooks)` -- pas de contrainte sur les diagonales. Le nombre de solutions est $N! = 8! = 40\,320$.

```python
def solve_nrooks_cpsat(n, enumerate_all=True):
    model = cp_model.CpModel()
    rooks = [model.NewIntVar(0, n - 1, f'r_{i}') for i in range(n)]
    model.AddAllDifferent(rooks)
    solver = cp_model.CpSolver()
    solver.parameters.enumerate_all_solutions = enumerate_all
    collector = SolutionCounter()
    solver.Solve(model, collector)
    return collector.count
```

</details>

### Exercice 2 : seuil de croisement backtracking / min-conflicts

**Enonce** : trouvez experimentalement le plus petit $N$ pour lequel min-conflicts est systematiquement plus rapide que le backtracking FC+MRV.

1. Testez les deux approches pour $N = 8, 10, 12, 14, 16, 18, 20$
2. Moyennez sur 5 essais (min-conflicts est stochastique)
3. Tracez les courbes de temps et identifiez le point de croisement

In [ ]:
# Exercice 2 : trouver le seuil backtracking vs min-conflicts

# A COMPLETER
# test_sizes = [8, 10, 12, 14, 16, 18, 20]
# n_trials = 5
# bt_avg_times = []
# mc_avg_times = []
#
# for n in test_sizes:
#     # Backtracking FC (deterministe, 1 essai suffit)
#     t0 = time.time()
#     backtracking_fc(n)
#     bt_avg_times.append((time.time() - t0) * 1000)
#
#     # Min-conflicts (stochastique, moyenner)
#     mc_t = []
#     for trial in range(n_trials):
#         t0 = time.time()
#         min_conflicts(n)
#         mc_t.append((time.time() - t0) * 1000)
#     mc_avg_times.append(np.mean(mc_t))

### Exercice 3 : brisure de symetrie avec CP-SAT

**Enonce** : le probleme des N-Reines admet des symetries (rotation de 90/180/270 degres, reflexions horizontale et verticale). On peut reduire l'espace de recherche en ajoutant des **contraintes de brisure de symetrie**.

1. Ajoutez la contrainte `queens[0] < queens[N-1]` (elimine la reflexion horizontale)
2. Ajoutez `queens[0] < N // 2` (elimine les rotations de 180 degres)
3. Comptez les solutions "fondamentales" (uniques sous symetrie) pour $N = 8$
4. Verifiez : il devrait y en avoir **12** solutions fondamentales

In [ ]:
# Exercice 3 : brisure de symetrie

# A COMPLETER
# def solve_nqueens_symmetry(n):
#     model = cp_model.CpModel()
#     queens = [model.NewIntVar(0, n - 1, f'q_{i}') for i in range(n)]
#
#     model.AddAllDifferent(queens)
#     model.AddAllDifferent([queens[i] + i for i in range(n)])
#     model.AddAllDifferent([queens[i] - i for i in range(n)])
#
#     # Brisure de symetrie
#     model.Add(queens[0] < queens[n - 1])  # elimine reflexion
#     model.Add(queens[0] < n // 2)          # elimine rotation 180
#
#     # Enumerer
#     ...

<details>
<summary><b>Indice exercice 3</b></summary>

Pour $N = 8$, il y a 92 solutions et 12 solutions fondamentales. Attention : les deux contraintes de symetrie proposees ne sont pas suffisantes pour isoler exactement les 12 fondamentales (il faudrait toutes les symetries du carre). Mais elles reduisent fortement le nombre de solutions a enumerer.

Pour obtenir exactement 12, il faut considerer le groupe diedre $D_4$ (8 symetries du carre) et n'enumerer qu'un representant par orbite. C'est un exercice plus avance.

</details>

---

## Recapitulatif

### Ce que nous avons appris

| Concept | Description | Section |
|---------|-------------|--------|
| Modelisation CSP | Variables = colonnes, valeurs = lignes, contraintes = no-attack | 2 |
| Backtracking | Recherche systematique avec verification incrementale | 3.1 |
| MRV | Heuristique fail-first pour le choix de variable | 3.2 |
| Forward Checking | Propagation des contraintes pour detection precoce d'echec | 3.3 |
| Min-Conflicts | Recherche locale quasi-lineaire mais incomplete | 4 |
| CP-SAT | Solveur industriel avec contraintes globales et enumeration | 5 |

### Tableau de synthese des approches

| Critere | Backtracking | Min-Conflicts | CP-SAT |
|---------|-------------|---------------|--------|
| Completude | Oui | Non | Oui |
| Complexite N-Reines | Exponentielle | Quasi-lineaire | Variable |
| Enumeration | Possible (lent) | Non | Oui (natif) |
| Difficulte d'implementation | Moyenne | Facile | Facile (declaratif) |
| Scalabilite | Faible (N < 30) | Excellente (N > 10^6) | Bonne (N > 10^3) |
| Pedagogie | Comprendre les CSP | Comprendre la recherche locale | Utiliser un outil pro |

### Pour aller plus loin

- [App-2 : Graph Coloring](App-2-GraphColoring.ipynb) -- un autre probleme CSP classique avec applications cartographiques
- [Search-7 : CSP Consistency](../Foundations/Search-7-CSP-Consistency.ipynb) -- propagation de contraintes (AC-3, MAC)
- [Search-8 : CSP Advanced](../Foundations/Search-8-CSP-Advanced.ipynb) -- contraintes globales, Large Neighborhood Search

### References

- Russell, S. & Norvig, P. *Artificial Intelligence: A Modern Approach*, 4th ed., Chapitre 6
- Minton, S., Johnston, M.D., Philips, A.B. & Laird, P. (1992). *Min-Conflicts: A Simple, Complete, Backtrack-Free Search Procedure*. Artificial Intelligence, 58(1-3)
- [OR-Tools CP-SAT Documentation](https://developers.google.com/optimization/cp/cp_solver)

---

**Navigation** : [Index](../README.md) | [Foundations](../Foundations/README.md) | [App-2 GraphColoring >>](App-2-GraphColoring.ipynb)